This workbook is a bit of quick walkthrough of how to use ghidra-bridge/ghidra-notebook. We'll write another one that actually walks through Ghidra use.

In [ ]:
# setup code
import ipywidgets

Once you've got the bridge connected, the flat API should be available in the globals. Let's see what program you've got open in Ghidra.

In [ ]:
print(currentProgram)

Let's do something a bit fancier. Let's get the list of functions in that program. I wonder how, though...

Let's start with a list of what's available on currentProgram:

In [ ]:
# dir will include the attributes and functions of the bridged object in the Ghidra python environment
dir(currentProgram)

getFunctionManager looks like it's interesting. Can we get some more info on that? 

In [ ]:
# help will attempt to get the Ghidra documentation if possible
help(currentProgram.getFunctionManager())


Great! getFunctions() sounds like what we want. Let's do it!

In [ ]:
%time list(currentProgram.getFunctionManager().getFunctions(True))

Wow, that was pretty slow - GhidraBridge has to pump multiple requests for each function, so it can take a while. Luckily, the bridge.remote_eval() function lets you run list/dictionary-comprehensions that can be a bit faster. We really just want the name of the function and the address, so let's try that.

In [ ]:
# the bridge is available in the globals as "bridge". 
# Note that remote_eval takes a string.
%time func_tuple_list = bridge.remote_eval("[(func.getEntryPoint().getOffset(), func.getName()) for func in currentProgram.getFunctionManager().getFunctions(True)]")
func_dict = { f"{name} ({hex(address)})": (address, name) for address, name in func_tuple_list}

Yay, way faster. Now let's pick a function to look at with a widget:

In [ ]:
dropD = ipywidgets.widgets.Dropdown(
 options=sorted(func_dict.keys()),
 description="Function name:",
 disabled=False,
 )
display(dropD)

Let's get the decompilation of the function.

In [ ]:
function_address = func_dict[dropD.value][0]
function = currentProgram.getFunctionManager().getFunctionAt(currentProgram.parseAddress(hex(function_address))[0])

from ghidra.app.decompiler import DecompInterface
from ghidra.util.task import ConsoleTaskMonitor

ifc = DecompInterface()
ifc.openProgram(currentProgram)

# decompile the function and print the pseudo C
results = ifc.decompileFunction(function, 0, ConsoleTaskMonitor())
print(results.getDecompiledFunction().getC())
